### Import

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier
from catboost import CatBoostClassifier

### Data Load

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/난임/train.csv').drop(columns=['ID'])
test = pd.read_csv('/content/drive/MyDrive/난임/test.csv').drop(columns=['ID'])

In [ ]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Train data 전처리

In [ ]:
import pandas as pd

# 매핑할 딕셔너리
mapping_dict = {'ICSI / BLASTOCYST:IVF / BLASTOCYST':'ICSI / BLASTOCYST :IVF / BLASTOCYST'}

# 매핑 적용
X['특정 시술 유형'] = X['특정 시술 유형'].apply(lambda x : mapping_dict.get(x,x))


In [ ]:
X['특정 시술 유형'] = X['특정 시술 유형'].fillna('GIFT')
X['특정 시술 유형'].isnull().sum()

0

In [ ]:
drop_list = ['난자 채취 경과일', '난자 해동 경과일', '배아 해동 경과일', '임신 시도 또는 마지막 임신 경과 연수', '불임 원인 - 여성 요인']

X[drop_list].isnull().sum()

,0
난자 채취 경과일,57488
난자 해동 경과일,254915
배아 해동 경과일,215982
임신 시도 또는 마지막 임신 경과 연수,246981
불임 원인 - 여성 요인,0


In [ ]:
fillna_list = ['착상 전 유전 검사 사용 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 혼합 경과일', '배아 이식 경과일','난자 채취 경과일', '난자 해동 경과일', '배아 해동 경과일', '임신 시도 또는 마지막 임신 경과 연수', '불임 원인 - 여성 요인' ]

for col in fillna_list:
    # '시술 유형'이 'DI'인 경우 해당 컬럼의 결측치를 -1로 채움
    X.loc[X['시술 유형'] == 'DI', col] = X.loc[X['시술 유형'] == 'DI', col].fillna(-1)

    # '시술 유형'이 'DI'가 아닌 경우 해당 컬럼의 결측치를 0으로 채움
    X.loc[X['시술 유형'] != 'DI', col] = X.loc[X['시술 유형'] != 'DI', col].fillna(0)


In [ ]:
X.fillna(-1,inplace = True)

### Data Pre-processing

In [ ]:
numeric_columns = [
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    '배아 이식 경과일',
    '난자 혼합 경과일'
]

In [ ]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부",
    '배아 생성 주요 이유'
]

In [ ]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)

In [ ]:
# X['미세주입 성공률']=X["미세주입에서 생성된 배아 수"]/X["미세주입된 난자 수"]
# X['난자 활용률']=X['이식된 배아 수']/X['수집된 신선 난자 수']
X['미세주입 성공률'] = np.where(
    X["미세주입된 난자 수"] > 0,  # 조건
    X["미세주입에서 생성된 배아 수"] / X["미세주입된 난자 수"],  # 조건이 True일 때
    0  # 조건이 False일 때
)

# 난자 활용률 계산
X['난자 활용률'] = np.where(
    X['수집된 신선 난자 수'] > 0,  # 조건
    X['이식된 배아 수'] / X['수집된 신선 난자 수'],  # 조건이 True일 때
    0  # 조건이 False일 때
)

tmp = pd.DataFrame()
translate_list=[
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태"
]
mapping_dict = {'1': 1, '0': 0}
for column in translate_list:
    tmp[f'{column}'] = X[column].map(mapping_dict).astype(int)

men_list=[
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "불임 원인 - 남성 요인",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태"
]

women_list=[
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 난관 질환",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증"
]

X['남성_불임_원인_합계'] = X[men_list].astype(int).sum(axis=1)

# 여성 불임 원인 합계
X['여성_불임_원인_합계'] = X[women_list].astype(int).sum(axis=1)

# 전체 불임 원인 합계
X['전체_불임_원인_합계'] = X[translate_list].astype(int).sum(axis=1)
X

,시술 시기 코드,시술 당시 나이,임신 시도 또는 마지막 임신 경과 연수,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,착상 전 유전 진단 사용 여부,...,난자 채취 경과일,난자 해동 경과일,난자 혼합 경과일,배아 이식 경과일,배아 해동 경과일,미세주입 성공률,난자 활용률,남성_불임_원인_합계,여성_불임_원인_합계,전체_불임_원인_합계
0,TRZKPL,만18-34세,0.0,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.800,0.285714,1,1,2
1,TRYBLT,만45-50세,0.0,IVF,ICSI,0,알 수 없음,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0,0,0
2,TRVNRY,만18-34세,0.0,IVF,IVF,1,기록되지 않은 시행,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.000,0.250000,1,0,1
3,TRJXFG,만35-37세,0.0,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,1,0,1
4,TRVNRY,만18-34세,0.0,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,1.000,0.285714,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256346,TRYBLT,만18-34세,0.0,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.625,0.181818,0,0,1
256347,TRYBLT,만38-39세,0.0,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,1.000,0.400000,1,0,1
256348,TRVNRY,만35-37세,0.0,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,1.000,0.200000,1,1,2
256349,TRZKPL,만38-39세,0.0,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.000,0.666667,0,0,1


In [ ]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encode = X.copy()
X_train_encode[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns]).astype(float)

In [ ]:
# 치우친 데이터를 정규 분포에 가깝게 만들기 위해 로그 변환 진행
# 로그 변환

# 로그 변환을 적용할 컬럼 선택 (0 이상이어야 하므로 +1 추가)
log_transform_cols = X_train_encode[numeric_columns]

X_train_log = X_train_encode.copy()
for col in log_transform_cols:
    X_train_log[col] = np.log1p(X_train_log[col]+1)  # log1p = log(x + 1), 0 값도 변환 가능

In [ ]:
X_train_log.describe()

,시술 시기 코드,시술 당시 나이,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,착상 전 유전 진단 사용 여부,남성 주 불임 원인,...,신선 배아 사용 여부,기증 배아 사용 여부,대리모 여부,PGD 시술 여부,PGS 시술 여부,미세주입 성공률,난자 활용률,남성_불임_원인_합계,여성_불임_원인_합계,전체_불임_원인_합계
count,256351.000000,256351.000000,256351.000000,256351.000000,256351.000000,256351.000000,256351.000000,256351.000000,256351.000000,256351.000000,...,256351.000000,256351.000000,256351.000000,256351.000000,256351.000000,256351.000000,256351.000000,256351.000000,256351.000000,256351.000000
mean,2.947923,1.302488,0.975459,9.963780,0.771286,0.724608,0.276827,0.059684,0.061548,0.028516,...,0.868801,0.058670,0.053173,0.057581,0.056606,0.348861,0.168545,0.416051,0.253293,1.092299
std,2.009923,1.392181,0.154720,6.262052,0.420005,1.284040,0.499276,0.324350,0.326867,0.166441,...,0.403816,0.322969,0.315321,0.321477,0.320130,0.390800,0.228663,0.539940,0.487508,0.857048
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,3.000000,1.000000,1.000000,12.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,1.000000
75%,5.000000,2.000000,1.000000,14.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.727273,0.222222,1.000000,0.000000,1.000000
max,6.000000,6.000000,1.000000,22.000000,1.000000,3.000000,2.000000,2.000000,2.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,1.250000,3.000000,5.000000,4.000000,8.000000


## test 데이터 전처리

In [ ]:
# 매핑할 딕셔너리
mapping_dict = {'ICSI / BLASTOCYST:IVF / BLASTOCYST':'ICSI / BLASTOCYST :IVF / BLASTOCYST'}

# 매핑 적용
test['특정 시술 유형'] = test['특정 시술 유형'].apply(lambda x : mapping_dict.get(x,x))

test['특정 시술 유형'] = test['특정 시술 유형'].fillna('GIFT')
test['특정 시술 유형'].isnull().sum()

drop_list = ['난자 채취 경과일', '난자 해동 경과일', '배아 해동 경과일', '임신 시도 또는 마지막 임신 경과 연수', '불임 원인 - 여성 요인']

# test.drop(drop_list, axis = 1, inplace = True)

fillna_list = ['착상 전 유전 검사 사용 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 혼합 경과일', '배아 이식 경과일','난자 채취 경과일', '난자 해동 경과일', '배아 해동 경과일', '임신 시도 또는 마지막 임신 경과 연수', '불임 원인 - 여성 요인' ]

for col in fillna_list:
    # '시술 유형'이 'DI'인 경우 해당 컬럼의 결측치를 -1로 채움
    test.loc[test['시술 유형'] == 'DI', col] = test.loc[test['시술 유형'] == 'DI', col].fillna(-1)

    # '시술 유형'이 'DI'가 아닌 경우 해당 컬럼의 결측치를 0로 채움
    test.loc[test['시술 유형'] != 'DI', col] = test.loc[test['시술 유형'] != 'DI', col].fillna(0)

# test[['난자 혼합 경과일', '배아 이식 경과일']].fillna(0, inplace = True)

test.fillna(-1,inplace = True)

<ipython-input-18-3ec864789101>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[['난자 혼합 경과일', '배아 이식 경과일']].fillna(0, inplace = True)


In [ ]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    test[col] = test[col].astype(str)

In [ ]:
# X['미세주입 성공률']=X["미세주입에서 생성된 배아 수"]/X["미세주입된 난자 수"]
# X['난자 활용률']=X['이식된 배아 수']/X['수집된 신선 난자 수']
test['미세주입 성공률'] = np.where(
    test["미세주입된 난자 수"] > 0,  # 조건
    test["미세주입에서 생성된 배아 수"] / test["미세주입된 난자 수"],  # 조건이 True일 때
    0  # 조건이 False일 때
)

# 난자 활용률 계산
test['난자 활용률'] = np.where(
    test['수집된 신선 난자 수'] > 0,  # 조건
    test['이식된 배아 수'] / test['수집된 신선 난자 수'],  # 조건이 True일 때
    0  # 조건이 False일 때
)

tmp = pd.DataFrame()
translate_list=[
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태"
]
mapping_dict = {'1': 1, '0': 0}
for column in translate_list:
    tmp[f'{column}'] = test[column].map(mapping_dict).astype(int)

men_list=[
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "불임 원인 - 남성 요인",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태"
]

women_list=[
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 난관 질환",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증"
]

test['남성_불임_원인_합계'] = test[men_list].astype(int).sum(axis=1)

# 여성 불임 원인 합계
test['여성_불임_원인_합계'] = test[women_list].astype(int).sum(axis=1)

# 전체 불임 원인 합계
test['전체_불임_원인_합계'] = test[translate_list].astype(int).sum(axis=1)
test

,시술 시기 코드,시술 당시 나이,임신 시도 또는 마지막 임신 경과 연수,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,착상 전 유전 진단 사용 여부,...,난자 채취 경과일,난자 해동 경과일,난자 혼합 경과일,배아 이식 경과일,배아 해동 경과일,미세주입 성공률,난자 활용률,남성_불임_원인_합계,여성_불임_원인_합계,전체_불임_원인_합계
0,TRYBLT,만35-37세,0.0,IVF,IVF,1,기록되지 않은 시행,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0,1
1,TRDQAZ,만18-34세,0.0,IVF,IVF,1,기록되지 않은 시행,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1,0,1
2,TRCMWS,만40-42세,0.0,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.555556,0.333333,0,0,0
3,TRJXFG,만40-42세,0.0,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,1.000000,1.000000,0,0,1
4,TRJXFG,만35-37세,0.0,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,1.000000,0.200000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90062,TRDQAZ,만18-34세,0.0,IVF,ICSI:ICSI,1,기록되지 않은 시행,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0,0,0
90063,TRYBLT,만43-44세,0.0,IVF,Unknown,0,알 수 없음,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0,0
90064,TRVNRY,만18-34세,0.0,IVF,IVF,0,알 수 없음,1.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.000000,0.000000,0,2,2
90065,TRCMWS,만43-44세,0.0,IVF,Unknown,0,알 수 없음,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.000000,0.000000,0,0,0


In [ ]:
X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [ ]:
# 로그 변환을 적용할 컬럼 선택 (0 이상이어야 하므로 +1 추가)
log_transform_cols = X_test_encoded[numeric_columns]

X_test_log = X_test_encoded.copy()
for col in log_transform_cols:
    X_test_log[col] = np.log1p(X_test_log[col]+1)  # log1p = log(x + 1), 0 값도 변환 가능

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_log)  # 훈련 데이터 변환 (fit & transform)
X_test_scaled = scaler.transform(X_test_log)  # 테스트 데이터 변환 (transform만

### Train

In [ ]:
import optuna
import xgboost as xgb
import warnings
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

# ✅ 경고 메시지 무시 설정
warnings.simplefilter("ignore")

# ✅ Optuna를 사용하여 하이퍼파라미터 최적화 (CV 활용)
def objective(trial):
    print(f"🔄 현재 시도 번호: {trial.number + 1}/20")  # ✅ 현재 반복 횟수 출력

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 300),
        "max_leaves": trial.suggest_int("max_leaves", 20, 40),
        "min_child_weight": trial.suggest_float("min_child_weight", 1.0, 7.0),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "subsample": trial.suggest_float("subsample", 0.8, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.6, 0.9),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 0.7),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 0.1),
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "use_label_encoder": False,
        "random_state": 42,
        "verbosity": 0
    }

    # ✅ K-Fold Cross Validation 설정 (5-Fold 사용)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    auc_scores = []

    for train_idx, valid_idx in skf.split(X_train_scaled, y):
        X_train_fold, X_valid_fold = X_train_scaled[train_idx], X_train_scaled[valid_idx]
        y_train_fold, y_valid_fold = y[train_idx], y[valid_idx]

        # ✅ XGBoost의 DMatrix 변환
        dtrain = xgb.DMatrix(X_train_fold, label=y_train_fold)
        dvalid = xgb.DMatrix(X_valid_fold, label=y_valid_fold)

        # ✅ Early Stopping 적용
        evals = [(dtrain, "train"), (dvalid, "valid")]
        model = xgb.train(
            params,
            dtrain,
            num_boost_round=300,  # 최대 300번 반복
            evals=evals,
            early_stopping_rounds=30,  # ✅ early stopping 적용
            verbose_eval=False
        )

        y_pred_proba = model.predict(dvalid)
        auc_scores.append(roc_auc_score(y_valid_fold, y_pred_proba))

    return np.mean(auc_scores)  # ✅ 평균 ROC-AUC 반환

# ✅ Optuna 실행 (반복 횟수 줄임)
study = optuna.create_study(direction="maximize")  # ROC-AUC 점수 최대화
study.optimize(objective, n_trials=20)  # ✅ 20번 반복 (기존 50 → 20)

# ✅ 최적 파라미터 출력
best_params_optuna = study.best_params
print("\n🔥 Optuna가 찾은 최적 XGBoost 하이퍼파라미터:")
print(best_params_optuna)

[I 2025-02-23 12:43:40,005] A new study created in memory with name: no-name-d3b2e567-9e2e-4054-a7e1-84bdccf8c956


🔄 현재 시도 번호: 1/20


[I 2025-02-23 12:45:23,559] Trial 0 finished with value: 0.7365128665607689 and parameters: {'n_estimators': 262, 'max_leaves': 28, 'min_child_weight': 1.0572740795305589, 'learning_rate': 0.013407187529942093, 'subsample': 0.8428318852847656, 'colsample_bylevel': 0.6798653094729662, 'colsample_bytree': 0.45552956660949206, 'reg_alpha': 0.43750191833912366, 'reg_lambda': 0.09269891559554946}. Best is trial 0 with value: 0.7365128665607689.


🔄 현재 시도 번호: 2/20


[I 2025-02-23 12:46:54,501] Trial 1 finished with value: 0.7397621150448053 and parameters: {'n_estimators': 297, 'max_leaves': 20, 'min_child_weight': 2.5359736802781034, 'learning_rate': 0.047450161665873195, 'subsample': 0.9491234365514285, 'colsample_bylevel': 0.6958034068211674, 'colsample_bytree': 0.5967135373350285, 'reg_alpha': 0.1021015302346013, 'reg_lambda': 0.0914645192065999}. Best is trial 1 with value: 0.7397621150448053.


...

🔄 현재 시도 번호: 19/20


[I 2025-02-23 13:07:05,046] Trial 18 finished with value: 0.7399131419927719 and parameters: {'n_estimators': 238, 'max_leaves': 28, 'min_child_weight': 1.8710114904789075, 'learning_rate': 0.05925350158045113, 'subsample': 0.8731668334175986, 'colsample_bylevel': 0.8505298214963752, 'colsample_bytree': 0.6586445248864716, 'reg_alpha': 0.9976420588418817, 'reg_lambda': 0.010161831971838677}. Best is trial 11 with value: 0.7399152472993115.


🔄 현재 시도 번호: 20/20


[I 2025-02-23 13:08:40,753] Trial 19 finished with value: 0.7400638009141656 and parameters: {'n_estimators': 275, 'max_leaves': 29, 'min_child_weight': 1.6755526076747729, 'learning_rate': 0.057179020253657876, 'subsample': 0.8690146464947366, 'colsample_bylevel': 0.8566469459038856, 'colsample_bytree': 0.6634748922492971, 'reg_alpha': 0.9946503888684214, 'reg_lambda': 0.030192073151510943}. Best is trial 19 with value: 0.7400638009141656.



🔥 Optuna가 찾은 최적 XGBoost 하이퍼파라미터:
{'n_estimators': 275, 'max_leaves': 29, 'min_child_weight': 1.6755526076747729, 'learning_rate': 0.057179020253657876, 'subsample': 0.8690146464947366, 'colsample_bylevel': 0.8566469459038856, 'colsample_bytree': 0.6634748922492971, 'reg_alpha': 0.9946503888684214, 'reg_lambda': 0.030192073151510943}


In [ ]:
# ✅ 최적 모델 생성 후 학습 (fit) 수행
best_model = xgb.XGBClassifier(**best_params_optuna, use_label_encoder=False, eval_metric="auc")
best_model.fit(X_train_scaled, y)  # ✅ 모델 학습 필수

# ✅ 학습된 모델로 테스트 데이터 예측
y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]

### Submission

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/난임/sample_submission.csv')
sample_submission['probability'] = y_pred_proba

In [ ]:
sample_submission.to_csv('/content/drive/MyDrive/난임/final_code.csv', index=False)